# sumaryzacja tekstu

In [1]:
from google.colab import drive


drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
!nvidia-smi

Sat Jan 31 15:08:11 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install "datasets<3.0.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.6.1 which is incompatible.


In [4]:
!pip install bert-extractive-summarizer

In [5]:
!pip install pytorch-ignite
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.8/343.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 23.5 MB/s eta 0:00:00


## importowanie datesetu

In [6]:
from datasets import load_dataset


ds = load_dataset("EdinburghNLP/xsum", trust_remote_code=True)
ds= ds.filter(lambda example: len(example["document"]) < 1500)
split_result = ds["test"].train_test_split(test_size=2, seed=42)
shots = split_result["test"]
ds["test"] = split_result["train"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Filter:   0%|          | 0/204045 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11332 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [7]:
print(ds)
print(shots)

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 84554
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 4705
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 4689
    })
})
Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 2
})


In [8]:
from summarizer import Summarizer
model_bertsum = Summarizer()

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
from ignite.metrics import Rouge

m = Rouge(variants=["L", 1], multiref="best")

candidate = "the cat is not there".split()
references = [
    "the cat is on the mat".split(),
    "there is a cat on the mat".split()
]

m.update(([candidate], [references]))

print(m.compute())

{'Rouge-L-P': 0.6, 'Rouge-L-R': 0.5, 'Rouge-L-F': 0.5, 'Rouge-1-P': 0.8, 'Rouge-1-R': 0.5714285714285714, 'Rouge-1-F': 0.5714285714285714}


In [10]:
num_workers = 1
batch_size = 1

In [11]:
from torch.utils.data import DataLoader
import torch
test_dataset = ds["test"].select(range(50))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
print(test_dataset)


Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 50
})


In [12]:
import string


def clean(text):
    if not text:
        return ["."]
    text = text.lower().translate(
        str.maketrans(string.punctuation, " " * len(string.punctuation))
    )
    return text.split()


def rouge_score_loss(
    batch,
    model,
    bert: bool,
    tokenizer: any,
    prompt=None,
    promptsuff=None,
    m=Rouge(variants=["L", 2], multiref="best"),
):
    if bert:
        predictions = [
            model(doc, ratio=0.1) for doc in batch["document"]
        ]
    else:
        docs = [doc for doc in batch["document"]]

        prompted = [
            f"{prompt}{doc}{promptsuff}" for doc in batch["document"]
        ]
        inputs = tokenizer(
            prompted,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=4096,
        )

        device = next(model.parameters()).device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        summary_ids = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=50,
            do_sample=True,
            top_k=0,
            temperature=0.5,
        )

        input_length = inputs["input_ids"].shape[1]
        new_tokens = summary_ids[:, input_length:]

        predictions = tokenizer.batch_decode(
            new_tokens, skip_special_tokens=True
        )

    print(f"Doc: {batch['document'][0][:100]}")
    print(f"Prediction: {predictions[0]}")
    print(f"Target: {batch['summary'][0]}")

    predictions = [
        clean(pred) if pred.strip() else "." for pred in predictions
    ]

    targets = [[clean(ref)] for ref in batch["summary"]]
    m.update((predictions, targets))
    print(m.compute())
    return m.compute()

In [13]:
from matplotlib import pyplot as plt
import numpy as np

def plot_data(data, labels=['Rouge-1-P', 'Rouge-1-R', 'Rouge-1-F','Rouge-2-P', 'Rouge-2-R', 'Rouge-2-F',  'Rouge-L-P', 'Rouge-L-R', 'Rouge-L-F', ], models = ['Qwen 1.5 0.5B - LoRa', 'Qwen 1.5 0.5B - few shot 2', 'Qwen 1.5 0.5B - few shot 5', 'Qwen 1.5 0.5B - zero shot', 'Bertsum - bert-base-uncased', 'Qwen 2.5 3B - zero shot']):
    x = np.arange(len(labels))
    width = 0.2

    plt.figure(figsize=(10, 6))
    for i, row in enumerate(data):
        offset = x + (i * width)
        plt.bar(offset, row, width=width, label=models[i])
    center_offset = ((len(data) - 1) * width) / 2
    plt.xticks(x + center_offset, labels)

    plt.ylim(0, 1)
    plt.ylabel('Score')
    plt.title('ROUGE Score Comparison')
    plt.legend()
    plt.show()

In [14]:
def batch_to_device(batch: dict, device) -> dict:
    return {k: v.to(device) for k, v in batch.items()}

In [15]:
import torch
import torch.nn as nn
from torch import Tensor
def calc_loss_batch(batch: dict, model: nn.Module) -> Tensor:
    logits = model(batch['input_ids'])
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), batch['labels'].flatten())
    return loss

In [16]:
import numpy as np
def calc_loss_loader(
    data_loader,
    model,
    device,
    bert: bool,
    tokenizer: any,
    prompt=None,
    promptsuff=None,
) -> float:
    results = []

    for i, batch in enumerate(data_loader):

        m = Rouge(variants=[1,2,"L"], multiref="best")

        ret = rouge_score_loss(
            batch,
            model,
            bert,
            tokenizer,
            prompt,
            promptsuff,
            m,
        )
        results.append(list(ret.values()))

    results = np.array(results)
    return np.mean(results, axis=0)


In [17]:
def evaluate_model(
    model: nn.Module,
    loader: DataLoader,
    device,
    bert: bool,
    tokenizer: any,
    prompt: str = "",
    promptsuff: str = "",
) -> tuple[float, float]:
    if not bert:
        model.eval()

    with torch.no_grad():
        loss = calc_loss_loader(
            loader,
            model,
            device,
            bert=bert,
            tokenizer=tokenizer,
            prompt=prompt,
            promptsuff=promptsuff,
        )

    return loss




In [18]:
from torch import nn
data_bertsum = evaluate_model(model_bertsum, test_loader, device='cuda', bert=True, tokenizer=None)
print(data_bertsum)



Doc: Midfielder Doughty stabbed home in the 94th minute after Millwall failed to clear a free-kick.
Lee G
Prediction: Midfielder Doughty stabbed home in the 94th minute after Millwall failed to clear a free-kick. "I thought we were outstanding in the first half and I think in the second half even though Swindon had all the football.
Target: Michael Doughty earned a draw against Millwall in Swindon's first game since Luke Williams was named manager.
{'Rouge-1-P': 0.15384615384615385, 'Rouge-1-R': 0.3333333333333333, 'Rouge-1-F': 0.3333333333333333, 'Rouge-2-P': 0.0, 'Rouge-2-R': 0.0, 'Rouge-2-F': 0.0, 'Rouge-L-P': 0.10256410256410256, 'Rouge-L-R': 0.2222222222222222, 'Rouge-L-F': 0.2222222222222222}
Doc: The victim was attacked because her son had eloped with an upper-caste girl about a month ago.
She h
Prediction: The victim was attacked because her son had eloped with an upper-caste girl about a month ago. On Wednesday, police arrested five people, including the eloped girl's parents.

In [19]:
def generate_prompt(tokenizer, size):
    prompt = ""
    shots = ds["train"].train_test_split(train_size=4, seed=35)["train"]
    shots_loader = DataLoader(shots, batch_size=100000, shuffle=True, num_workers=num_workers)
    for batch in shots_loader:
        for i in range(len(batch['document'])):
            prompt += "[USER] Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer."
            prompt += f'[TEXT]{batch['document'][i]}[AGENT]{batch['summary'][i]}'
            prompt += tokenizer.eos_token + "\n"
    prompt += "[USER] Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer.[TEXT]"
    return prompt


In [20]:
!pip uninstall -y unsloth unsloth_zoo

# 2. Install Unsloth Zoo DIRECTLY from Git (to get 'tiled_mlp')
!pip install --upgrade --no-cache-dir "unsloth_zoo @ git+https://github.com/unslothai/unsloth-zoo.git"

# 3. Install Unsloth main library from Git
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth-zoo.git to /tmp/pip-install-tets3zeh/unsloth-zoo_c025443ac2914bdfb366c136d822b04f
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth-zoo.git /tmp/pip-install-tets3zeh/unsloth-zoo_c025443ac2914bdfb366c136d822b04f
  Resolved https://github.com/unslothai/unsloth-zoo.git to commit 2a80d543b9e22f68e051e32029c8a47005102895
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 204.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 411.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 443.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 341.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-is7h0gw8/unsloth_3ecf7a2eae4448c495fb933f914f15e0
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-is7h0gw8/unsloth_3ecf7a2eae4448c495fb933f914f15e0
  Resolved https://github.com/unslothai/unsloth.git to commit 07a7ff47b1b2b37c088b8e0d7ed7bf8710d9aa22
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 10.0 MB/s eta 0:00:00
  Created wheel for unsloth: filename=unsloth-2026.1.4-py3-none-any.whl size=412718 sha256=fbaef3dd0450c7efe221b209fd4812ca202903ff5e756cad109306955224499a
  Stored in directory: /tmp/pip-ephem-wheel-cache-nxw9fr1l/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth


In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from unsloth import FastLanguageModel



/tmp/ipython-input-1311217971.py:2: UserWarning: WARNING: Unsloth should be imported before [transformers] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [22]:
import gc
try:
  del model
  del tokenizer3
  del model
except NameError:
  pass

gc.collect()

90

In [33]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen1.5-0.5B", max_seq_length=32000, load_in_4bit=False
)
prompt2 = generate_prompt(tokenizer, 2)
print(prompt2)
torch.cuda.empty_cache()
data_2 = evaluate_model(model, test_loader, device='cuda', bert=False, tokenizer=tokenizer, prompt=prompt2, promptsuff = "[AGENT]")

==((====))==  Unsloth 2026.1.4: Fast Qwen2 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Qwen/Qwen1.5-0.5B does not have a padding token! Will use pad_token = <|PAD_TOKEN|>.
[USER] Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer.[TEXT]The Hartswood Films production beat the likes of EastEnders, Strictly Come Dancing, The Great British Bake Off and Game of Thrones.
But its star, Benedict Cumberbatch, lost out on the best actor prize for a third time.
The honour went instead to Jason Watkins, for his role in The Lost Honour of Christopher Jefferies.
Backstage at the Sunday night ceremony , Sher

In [24]:

prompt5 = generate_prompt(tokenizer, 5)
torch.cuda.empty_cache()
data_5 = evaluate_model(model, test_loader, device='cuda', bert=False, tokenizer=tokenizer, prompt=prompt5, promptsuff = "[AGENT]")

[USER] Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer.[TEXT]The teenager was taken into custody following a police raid at his home in Croxteth, Liverpool, at 07:30 BST.
Detectives are investigating allegations that someone illegally accessed an Instagram account before attempting to blackmail its owner.
An online shopping account was also allegedly accessed in order to buy and steal goods.
Computer equipment and other electronic devices were seized at the house, and the boy was taken into custody for questioning.
Det Ch Supt Chris Green, head of Titan, the North West's organised crime unit, said: "Cyber-crime is increasingly common with criminals turning to the internet to commit the same kinds of offences as traditionally would be committed in the real world such as theft, blackmail and fraud.
"As we now all spend more and more of our daily lives online, we need to take the same precautions and care about our personal details as we would on 

In [25]:
prompt0 = "[USER] Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer.[TEXT]"
data_zero_shot = evaluate_model(
        model,
        test_loader,
        device="cuda",
        bert=False,
        tokenizer=tokenizer,
        prompt=prompt0,
        promptsuff = "[AGENT]"
    )


Doc: 5 January 2016 Last updated at 10:38 GMT
It is not known how the swan got trapped on the rooftop of 
Prediction:  [USER] How did the swan get trapped on the rooftop of Agatha's Boutique? The swan was trapped on the rooftop of Agatha's Boutique due to insufficient space to take off.

Can you provide any information on how long it took
Target: An RSPCA worker has rescued an adult male swan from the roof of a building in the centre of Lincoln.
{'Rouge-1-P': 0.11904761904761904, 'Rouge-1-R': 0.25, 'Rouge-1-F': 0.25, 'Rouge-2-P': 0.0, 'Rouge-2-R': 0.0, 'Rouge-2-F': 0.0, 'Rouge-L-P': 0.11904761904761904, 'Rouge-L-R': 0.25, 'Rouge-L-F': 0.25}
Doc: The 37-year-old, who had already quit one-day and Twenty20 cricket, will finish his international ca
Prediction:  It's been a long time since I've had a player like this. I've always had a lot of faith in him and he's been a very good servant to the club.[PLAYER] I've been a big fan of Sangakkara for
Target: Sri Lanka batsman Kumar Sangakkara w

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
from peft import PeftModel
model_path = './drive/MyDrive/models/ft'
ft_model = PeftModel.from_pretrained(model, model_path, max_sequence_length=32000, load_in_4bit=False)


In [28]:
data_ft = evaluate_model(ft_model, test_loader, 'cuda', bert=False, tokenizer = tokenizer, prompt = "[USER] Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer.[TEXT]", promptsuff = "[AGENT]")

Doc: Chief executive Karl Koehler is standing down and his replacement will not have a position on the ma
Prediction: Tata Steel has announced that it will cut 1,050 jobs in the UK and Ireland by the end of 2015. The move comes amid a global downturn in the steel industry. The company has said it will be
Target: Changes at the top of Tata Steel Europe raise doubts about the "long-term investment" of the company, according the steelworkers union, Community.
{'Rouge-1-P': 0.20512820512820512, 'Rouge-1-R': 0.34782608695652173, 'Rouge-1-F': 0.34782608695652173, 'Rouge-2-P': 0.05263157894736842, 'Rouge-2-R': 0.09090909090909091, 'Rouge-2-F': 0.09090909090909091, 'Rouge-L-P': 0.15384615384615385, 'Rouge-L-R': 0.2608695652173913, 'Rouge-L-F': 0.2608695652173913}
Doc: Firefighters were called to a home in Marlborough Road just after 22:00 GMT on Saturday after a fire
Prediction: A man has died after a house fire in Oxford. The fire broke out in a flat in Marlborough Road at about 22:30 GMT on 

In [ ]:
model_better, tokenizer_better = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen2.5-3B", max_seq_length=32000, load_in_4bit=False
)
print(prompt0)
torch.cuda.empty_cache()
data_better = evaluate_model(model_better, test_loader, device='cuda', bert=False, tokenizer=tokenizer, prompt=prompt2, promptsuff = "[AGENT]")

==((====))==  Unsloth 2026.1.4: Fast Qwen2 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
plot_data([data_ft, data_2,data_5, data_zero_shot, data_bertsum, model_better])


In [ ]:
text = """
BLEU: Measuring n-gram Precision
BLEU (Bilingual Evaluation Understudy) evaluates how closely a model-generated text matches a reference text, based on n-gram matching. Commonly used for machine translation, BLEU measures how well words or sequences of words (n-grams) in the generated text align with the reference text.

n-gram Precision: BLEU checks more than just individual word matches, looking at sequences of two or more words (n-grams). For example, 1-gram measures single word accuracy, while 2-gram captures two-word sequences.
Brevity Penalty: To avoid rewarding overly short responses, BLEU includes a penalty for sentences that are too brief. This ensures that short texts don’t artificially inflate their scores by matching only the most common words.
BLEU scores range from 0 to 1, with scores closer to 1 indicating a high similarity with the reference text. It is especially reliable when multiple reference texts are used.

ROUGE: Optimized for Summarization
ROUGE (Recall-Oriented Understudy for Gisting Evaluation) was developed to measure summarization accuracy, focusing on how well the generated text captures essential content from the reference text.

The metric offers several variations:

ROUGE-N: Measures n-gram overlap between generated and reference texts. ROUGE-1 (single-word matches) and ROUGE-2 (two-word matches) are commonly used.
ROUGE-L: Focuses on the longest common subsequence (LCS) between the generated and reference texts, which preserves the sentence structure and improves coherence.
ROUGE-W: A weighted version of ROUGE-L, emphasizing longer matching sequences to give more accurate results in summarization.
ROUGE is particularly effective for evaluating summaries, as it checks whether the generated text captures key information.
"""

In [ ]:

prompt = [
            f"{prompt}{doc}{promptsuff}" for doc in batch["document"]
]
inputs = tokenizer(
  prompted,
  return_tensors="pt",
  padding=True,
  truncation=True,
  max_length=4096,
)

device = next(model.parameters()).device
inputs = {k: v.to(device) for k, v in inputs.items()}

summary_ids = model.generate(
  inputs["input_ids"],
  attention_mask=inputs["attention_mask"],
  max_new_tokens=50,
  do_sample=True,
  top_k=0,
  temperature=0.5,
)

input_length = inputs["input_ids"].shape[1]
new_tokens = summary_ids[:, input_length:]

predictions = tokenizer.batch_decode(
  new_tokens, skip_special_tokens=True
)